In [ ]:
import numpy as np
import json
from cad_viewer_widget import CadViewer
from sidecar import Sidecar

names = ["box", "box1", "boxes", "edges", "faces", "hexapod", "hexapod_m", "vertices"]
objects = {}
states = {}

for name in names:
    with open(f"examples/{name}.json", "r") as fd:
        objects[name] = json.load(fd)

    with open(f"examples/{name}-states.json", "r") as fd:
        states[name] = json.load(fd)

In [ ]:
sc = Sidecar(title="Tests")

with sc:
    cv = CadViewer(
        cad_width=800, 
        height=600, 
        tree_width=250, 
        theme="light", 
        tools=True
    )

In [ ]:
name = "hexapod"

cv.add_shapes(
    objects[name],
    states[name],
    ortho=True,
    control="orbit",
    ticks=10,
    axes=True,
    axes0=False,
    grid=[True, False, False], 
    transparent=True,
    black_edges=False,
    edge_color="#707070",
    ambient_intensity=0.5,
    direct_intensity=0.3,
    timeit=False,
)

# Widget comms

In [ ]:
cv.widget.states

In [ ]:
cv.set_states({
    '/bottom/bottom_0': (1,0),
    '/bottom/top/top_0': [0,1],
})

In [ ]:
cv.set_states({
    '/bottom/bottom_0': (1,1),
    '/bottom/top/top_0': [1,1],
})

In [ ]:
p0 = cv.widget.position

In [ ]:
cv.widget.position = [960.01,0,0]
cv.widget.zoom = 0.5

In [ ]:
cv.widget.position = p0
cv.widget.zoom = 1

In [ ]:
cv.widget.ambient_intensity = 0.9
cv.widget.direct_intensity = 0.3

In [ ]:
cv.widget.ambient_intensity = 0.5
cv.widget.direct_intensity = 0.3

In [ ]:
ec = cv.widget.edge_color

In [ ]:
cv.widget.edge_color = "#ff0000"

In [ ]:
cv.widget.edge_color = ec

In [ ]:
# cv.widget.grid = [not g for g in cv.widget.grid]
cv.widget.axes = not cv.widget.axes
cv.widget.axes0 = not cv.widget.axes0
cv.widget.transparent = not cv.widget.transparent
cv.widget.black_edges = not cv.widget.black_edges

In [ ]:
cv.widget.tools = not cv.widget.tools

In [ ]:
cv.widget.ortho = not cv.widget.ortho

In [ ]:
cv.widget.zoom_speed = 5
cv.widget.pan_speed = 5
cv.widget.rotate_speed = 5

In [ ]:
cv.widget.zoom_speed = 1
cv.widget.pan_speed = 1
cv.widget.rotate_speed = 1

In [ ]:
cv.widget.lastPick

# Custom Messsage

In [ ]:
from cad_viewer_widget import show_msg

debug = show_msg()

In [ ]:
from ipywidgets import Output

def action(result):
    value = json.loads(result)["result"]
    with out:
        print(value)

out = Output()
out

In [ ]:
cv.execute("addShapes", None, callback=action)

In [ ]:
cv.get("viewer.camera.zoom", callback=debug)

In [ ]:
cv.execute("viewer.camera.position", "set", (4.3, 0, 0), update=True, callback=debug)

In [ ]:
cv.set("viewer.camera.zoom", 0.2, update=True, callback=debug)

In [ ]:
cv.get("viewer.camera.position", callback=action)

In [ ]:
cv.get_result()

In [ ]:
cv.get("viewer.scene.children[0].children[0].children[1]", callback=action)

In [ ]:
cv.execute("viewer", "setAxes", (True), update=True, callback=action)

In [ ]:
cv.get("viewer.scene.children[2]", callback=action)

In [ ]:
cv.zoom

In [ ]:
cv.widget.zoom = 0.5

In [ ]:
import json
print(json.dumps(cv.widget.states))

In [ ]:
States = {
  "unselected": 0,
  "selected": 1,
  "mixed": 2,
  "empty": 3
};

In [ ]:
def set_state(shape, mesh):
    if shape > 15 or mesh > 15 or shape < 0 or mesh < 0:
        raise ValueError("Values need to be between 0 and 15")
    return (shape << 4) + mesh

In [ ]:
def get_state(state):
    if state > 255 or state < 0:
        raise ValueError("Value needs to be between 0 and 255")
    return (state >> 4, state & 15)

In [ ]:
state = set_state(0,15)
state

In [ ]:
get_state(state)

# BACKUP

In [ ]:
import math

def rotate_x(vector, angle):
    angle = rad(angle)
    mat = np.array(
        [
            [1, 0, 0],
            [0, math.cos(angle), -math.sin(angle)],
            [0, math.sin(angle), math.cos(angle)],
        ]
    )
    return tuple(np.matmul(mat, vector))


def rad(deg):
    return deg / 180.0 * math.pi


def rotate_y(vector, angle):
    angle = rad(angle)
    mat = np.array(
        [
            [math.cos(angle), 0, math.sin(angle)],
            [0, 1, 0],
            [-math.sin(angle), 0, math.cos(angle)],
        ]
    )
    return tuple(np.matmul(mat, vector))


def rotate_z(vector, angle):
    angle = rad(angle)
    mat = np.array(
        [
            [math.cos(angle), -math.sin(angle), 0],
            [math.sin(angle), math.cos(angle), 0],
            [0, 0, 1],
        ]
    )
    return tuple(np.matmul(mat, vector))


def rotate(vector, angle_x=0, angle_y=0, angle_z=0):
    v = tuple(vector)
    if angle_z != 0:
        v = rotate_z(v, angle_z)
    if angle_y != 0:
        v = rotate_y(v, angle_y)
    if angle_x != 0:
        v = rotate_x(v, angle_x)
    return v